#### GPT 4o Few Shot assesesment
Qualitative assesment of fewshot and fewshot2 (are they really worse?)

Conclusion, with GPT 4o, zero shot actually performs best (perhaps overfiting less to the single example given)

In [2]:
import  pandas as pd
from IPython.display import display, HTML

In [9]:
# Load predictions
zeroshot = pd.read_csv('outputs/demographicseval_demographics-zeroshot_gpt-4o-2024-05-13_minc-40_maxc-4000_clean.csv')
fewshot = pd.read_csv('outputs/demographicseval_demographics-fewshot_gpt-4o-2024-05-13_minc-40_maxc-4000_clean.csv')
fewshot2 = pd.read_csv('outputs/demographicseval_demographics-fewshot2_gpt-4o-2024-05-13_minc-40_maxc-4000_clean.csv')

# Load only subset of columns
embeddings = pd.read_parquet('outputs/demographicseval_embeddings_minc-40_maxc-4000.parquet', 
                             columns=['content', 'start_char', 'end_char', 'pmcid', 'section_0', 'section_1'])

# Load annotations
annotations = pd.read_csv('annotations/combined_pd.csv').drop(columns=['Unnamed: 0'])
annotations.columns = annotations.columns.str.replace(' ', '_')

In [10]:
def _print_context(pmcid, annotation, predictions, embeddings):
    print("PMC ID: ", pmcid)
    subset_rows = ['count', 'diagnosis', 'group_name', 'subgroup_name',
       'male_count', 'female_count', 'age_mean', 'age_minimum', 'age_maximum',
       'age_median']
    
    pred_subset_cols = ['imaging_sample']
    
    if 'assesment_type' in predictions.columns:
        pred_subset_cols += ['assesment_type']
    
    annotation = annotation[(annotation.pmcid == pmcid)]
    annotation = annotation[subset_rows]

    pred = predictions[predictions.pmcid == pmcid]

    for start_char, g in pred.groupby('start_char'):
        print("Context: ")
        print(embeddings[(embeddings.pmcid == pmcid) & (embeddings.start_char == start_char)].content.values[0])

        print("Predictions: ")
        display(g[subset_rows + pred_subset_cols])

    print("Annotation: ")
    display(annotation)

In [16]:
z_mean = zeroshot.groupby('pmcid').apply(lambda x: x['count'].mean())
annot_mean = annotations.groupby('pmcid').apply(lambda x: x['count'].mean())
worst = ((z_mean-annot_mean) / annot_mean).abs().sort_values(ascending=False).head(10)

combined2 = pd.concat([zeroshot, fewshot, fewshot2], keys=['zero', 'few', 'few2'], names=['model'])

In [17]:
for i in worst.index:
    _print_context(i, annotations, combined2, embeddings)

PMC ID:  5598991
Context: 

### 1.2 Participants 
  
All patients were recruited from the outpatient clinic or inpatient wards of the Psychiatry Department of the First Affiliated Hospital of Kunming Medical University. Right-handed, 147 MDD patients (50 males and 97 females) met the criteria were recruited in the depression group. The inclusion criteria were as follows: ① The diagnosis of MDD was independently made by two experienced psychiatrists in accordance with the Diagnostic and Statistical Manual of Mental Disorders, fourth edition (DSM-IV, American Psychiatry Association, 1994), ② first episode without a history of antidepressants treatment, ③ be aged between 18–45 years, ④ the total score of 17-item Hamilton Depression Rating Scale (HDRS) was not less than 17, ⑤ right handedness, ⑥ the patients or their legal guardian signed the informed consent form. The exclusion criteria included the following items: ① having a history of Axis I psychiatric disorders. ② having a history of

count diagnosis group_name          subgroup_name  male_count  \
model                                                                      
zero  140  147.0       MDD   patients       depression group        50.0   
      141  130.0       NaN    healthy  healthy control group        49.0   
few   184  147.0       MDD   patients             depression        50.0   
      185  130.0       NaN    healthy                control        49.0   
few2  155  147.0       MDD   patients                    NaN        50.0   
      156  130.0       NaN    healthy                    NaN        49.0   

           female_count  age_mean  age_minimum  age_maximum  age_median  \
model                                                                     
zero  140          97.0       NaN         18.0         45.0         NaN   
      141          81.0       NaN          NaN          NaN         NaN   
few   184          97.0       NaN         18.0         45.0         NaN   
      185          81.0       NaN          NaN          NaN         NaN   
few2  155          97.0       NaN         18.0         45.0         NaN   
      156          81.0       NaN          NaN          NaN         NaN   

          imaging_sample assesment_type  
model                                    
zero  140            yes            NaN  
      141            yes            NaN  
few   184            yes            NaN  
      185            yes            NaN  
few2  155            NaN        imaging  
      156            NaN        imaging

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
196,58,early adult onset depression,patients,eod,NaN,NaN,NaN,NaN,NaN,NaN
197,62,later adult onset depression,patients,lod,NaN,NaN,NaN,NaN,NaN,NaN
198,60,NaN,healthy,young hc,NaN,NaN,NaN,NaN,NaN,NaN
199,52,NaN,healthy,old hc,NaN,NaN,NaN,NaN,NaN,NaN


PMC ID:  6016936
Context: 

## 2. Experimental procedures and results 
  
Predictions: 


...

,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
361,30,NaN,healthy,fmri experiment,9.0,21.0,23.0,NaN,NaN,NaN
362,9,NaN,healthy,tms-mep experiment,2.0,7.0,24.0,NaN,NaN,NaN


PMC ID:  8315277
Context: 

## Materials and Methods 
  
### Participants 
  
We recruited 104 patients with WMHs (39 with mild WMHs, 37 with moderate WMHs, and 28 with severe WMHs) and 37 HCs. The clinical manifestations of WMHs are heterogeneous, and the clinical symptoms of WMHs do not exhibit a pattern; therefore, there are no established diagnostic criteria. Consequently, the diagnostic criteria mainly depend on imaging features. As a result, the inclusion criteria for the WMHs group were as follows: (1) age between 40 and 80 years and (2) visible WMHs on T2 fluid-attenuated inversion recovery (T2 FLAIR). The exclusion criteria were as follows: (1) intracranial and extracranial stenosis >50%; (2) Trial of Org 10172 in Acute Stroke Treatment classification suggestive of cardiogenic stroke; (3) non-CSVD-related WMHs (e.g., multiple sclerosis); (4) mental disorders or alcohol addiction; (5) dementia or tumors; (6) intracranial hemorrhage; (7) significant hearing or visual impairment,

count      diagnosis group_name subgroup_name  male_count  \
model                                                                  
zero  233   39.0      mild WMHs   patients          mild         NaN   
      234   37.0  moderate WMHs   patients      moderate         NaN   
      235   28.0    severe WMHs   patients        severe         NaN   
      236   37.0        no WMHs    healthy           HCs         NaN   
few   285   39.0      mild WMHs   patients          mild         NaN   
      286   37.0  moderate WMHs   patients      moderate         NaN   
      287   28.0    severe WMHs   patients        severe         NaN   
      288   37.0            NaN    healthy           NaN         NaN   
few2  253   39.0      mild WMHs   patients          mild         NaN   
      254   37.0  moderate WMHs   patients      moderate         NaN   
      255   28.0    severe WMHs   patients        severe         NaN   
      256   37.0            NaN    healthy           NaN         NaN   

           female_count  age_mean  age_minimum  age_maximum  age_median  \
model                                                                     
zero  233           NaN       NaN         40.0         80.0         NaN   
      234           NaN       NaN         40.0         80.0         NaN   
      235           NaN       NaN         40.0         80.0         NaN   
      236           NaN       NaN          NaN          NaN         NaN   
few   285           NaN       NaN         40.0         80.0         NaN   
      286           NaN       NaN         40.0         80.0         NaN   
      287           NaN       NaN         40.0         80.0         NaN   
      288           NaN       NaN          NaN          NaN         NaN   
few2  253           NaN       NaN         40.0         80.0         NaN   
      254           NaN       NaN         40.0         80.0         NaN   
      255           NaN       NaN         40.0         80.0         NaN   
      256           NaN       NaN          NaN          NaN         NaN   

          imaging_sample assesment_type  
model                                    
zero  233            yes            NaN  
      234            yes            NaN  
      235            yes            NaN  
      236            yes            NaN  
few   285            yes            NaN  
      286            yes            NaN  
      287            yes            NaN  
      288            yes            NaN  
few2  253            NaN        imaging  
      254            NaN        imaging  
      255            NaN        imaging  
      256            NaN        imaging

Annotation: 


,count,diagnosis,group_name,subgroup_name,male_count,female_count,age_mean,age_minimum,age_maximum,age_median
370,104,White matter hyperintensities,patients,_,NaN,NaN,NaN,NaN,NaN,NaN
371,37,NaN,healthy,_,NaN,NaN,NaN,NaN,NaN,NaN
